In [3]:
import pickle as pkl
import pandas as pd

with open("torchxray-resnet-features.pkl", "rb") as f:
    features = pkl.load(f)

In [4]:
df = pd.DataFrame(features)
df.head()

,emb,subject,study,dicom
0,"[0.013232384, 0.00065970724, 0.06945538, 0.003...",10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014
1,"[0.0, 0.00084097317, 0.09649825, 2.6821714e-05...",10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab
2,"[0.00056263205, 0.0, 0.12521967, 0.0, 0.019728...",10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714
3,"[0.004915943, 0.0, 0.074994095, 0.0, 0.0108106...",10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818
4,"[0.036993176, 0.00011738534, 0.06852835, 0.0, ...",10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c


In [5]:
metadata = pd.read_csv("../../MIMIC-CXR-JPG/mimic-cxr-jpg/2.1.0/mimic-cxr-2.0.0-metadata.csv")
metadata.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN


In [7]:
# Join features with metadata on subject-subject_id, study-study_id and dicom-dicom_id

"""
emb        object
subject    object
study      object
dicom      object
dtype: object
dicom_id                                       object
subject_id                                      int64
study_id                                        int64
PerformedProcedureStepDescription              object
ViewPosition                                   object
Rows                                            int64
Columns                                         int64
StudyDate                                       int64
StudyTime                                     float64
ProcedureCodeSequence_CodeMeaning              object
ViewCodeSequence_CodeMeaning                   object
PatientOrientationCodeSequence_CodeMeaning     object
dtype: object
"""

df["subject_id"] = df["subject"].astype("int64")
df["study_id"] = df["study"].astype("int64")
df["dicom_id"] = df["dicom"].astype("object")
# drop subject, study, dicom
df = df.drop(columns=["subject", "study", "dicom"])

merged = df.merge(
    metadata,
    left_on=["subject_id", "study_id", "dicom_id"],
    right_on=["subject_id", "study_id", "dicom_id"],
    how="inner",
)

In [8]:
merged.head()

,emb,subject_id,study_id,dicom_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,"[0.013232384, 0.00065970724, 0.06945538, 0.003...",10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,"[0.0, 0.00084097317, 0.09649825, 2.6821714e-05...",10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
2,"[0.00056263205, 0.0, 0.12521967, 0.0, 0.019728...",10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN
3,"[0.004915943, 0.0, 0.074994095, 0.0, 0.0108106...",10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,CHEST (PORTABLE AP),AP,2906,2258,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,Erect
4,"[0.036993176, 0.00011738534, 0.06852835, 0.0, ...",10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,CHEST (PORTABLE AP),AP,3056,2544,21800805,234424.765,CHEST (PORTABLE AP),antero-posterior,NaN


In [ ]:
with open("resnet224-embeddings-metadata.pkl", "wb") as f:
    pkl.dump(merged, f)